In [1]:
import numpy as np
from random import shuffle
from math import log, floor
import pandas as pd
import tensorflow as tf
import tensorboard as tb
from keras import backend as K
from keras.models import *
from keras.layers import *
from keras.activations import *
from keras.callbacks import *
from keras.utils import *
from keras.layers.advanced_activations import *
# from keras.layers.advanced_activations import *
from keras import *
from keras.engine.topology import *
from keras.optimizers import *
import keras
# import pandas as pd
# import numpy as np
# import sklearn
import pickle
from keras.applications import *
from keras.preprocessing.image import *
# importing dependencies
import pandas as pd # data frame
import numpy as np # matrix math
from scipy.io import wavfile # reading the wavfile
from sklearn.utils import shuffle # shuffling of data
from random import sample # random selection
from tqdm import tqdm # progress bar
import matplotlib.pyplot as plt # to view graphs
import wave
from math import log, floor
# audio processing
from scipy import signal # audio processing
from scipy.fftpack import dct
import librosa # library for audio processing
import numpy as np
import pandas as pd
from sklearn.decomposition import *
from sklearn.cluster import KMeans
import sys, os
import tqdm ##
from tqdm import * ##
from xgboost.sklearn import XGBClassifier
from sklearn.utils import shuffle # shuffling of data
from random import sample # random selection
from tqdm import tqdm # progress bar
# audio processing
from scipy import signal # audio processing
from scipy.fftpack import dct
import librosa # library for audio processing
import xgboost as xgb
import lightgbm as lgb
import catboost as ctb
from keras.utils import *
from sklearn.ensemble import *
import pickle
from bayes_opt import BayesianOptimization
from logHandler import Logger
from utils import readCSV, getPath, writePickle,readPickle
from keras.regularizers import l2
from keras.callbacks import History ,ModelCheckpoint, EarlyStopping

/home/leoqaz12/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
folder = 'data/predict_test/' #共同predict對stacling verified data的結果
# acc_df = pd.read_csv('data/ens_unverified/validation_ACC_P1S1.csv') #accuracy csv
acc_df = pd.read_csv('data/predict_test/X_test_ACC.csv')

# acc_df.columns = ['model','csv_name','acc']
acc_df.columns = ['csv_name','acc']
acc_df = acc_df.filter(['csv_name','acc'])
# acc_df['csv_name'] = acc_df['csv_name'].str.replace('_unverified_','_')
files = os.listdir(folder)

ratio_all=0

df_dict = {}
for i,csv in enumerate(files):
    df_name = csv[:csv.rfind('_')]
#     print(df_name)
    if csv.endswith('ACC.csv'):
        continue
    
    ratio = acc_df[acc_df['csv_name'] == csv]['acc'].values[0]
    ratio_all += ratio
    
    
    if i==0:
        df = pd.read_csv(os.path.join(folder,csv))
        df = df.drop('fname',axis=1)
        df = np.array(df) * ratio
#         df1_ = pd.DataFrame(df1_)
#         df1_['acc'] = ratio
#         print(len(df1_))
#         df_dict[df_name] = df_dict[df_name].filter(['fname'])
#         df_dict[df_name] = pd.merge(df_dict[df_name],df1_,how='inner',right_index=True,left_index=True)
    else:
        df_ = pd.read_csv(os.path.join(folder,csv))
        df_ = df_.drop('fname',axis=1)
        df += np.array(df_) * ratio
#         df1_ = pd.DataFrame(df1_)
#         df1_['acc'] = ratio
# #         print(len(df1_))
#         df1_name = df1_name.filter(['fname'])
#         df1_ = pd.merge(df1_name, df1_,how='inner',right_index=True,left_index=True)
#         df_dict[df_name] = df_dict[df_name].append(df1_, ignore_index=True)

# for k,v in df_dict.items():
#     df_dict[k] = v.sort_values('fname')

# _ = list(df_dict.keys())
# sum_ = np.zeros((len(df_dict[_[0]]), 42))
# for k,v in df_dict.items():
#     sum_ += v[v.columns[1:]].values
# ratio_ = np.tile(sum_[:, -1], (41, 1)).T
test_X = df / ratio_all
print(sum(test_X[0]))
test_X.shape

1.000000006327188


(9400, 41)

In [3]:
reverse_dict = pickle.load(open('data/map_reverse.pkl' , 'rb'))

## Ensemble Result

In [4]:
name = pd.read_csv('data/sample_submission.csv')
X_name = name['fname'].tolist()
output = []

for i , d in enumerate(test_X):
    # print(i)
    # print(d)
    # print('\n')
    top3 = d.argsort()[-3:][::-1]
    result = [reverse_dict[x] for x in top3]
    s = ' '.join(result)

    output.append(s)
    
    
df = pd.DataFrame(output , columns = ['label'])
df.insert(0,'fname' , X_name)
df.to_csv('result/ens_ans.csv', index=False)
print(df.head(10))

          fname                                    label
0  00063640.wav  Computer_keyboard Shatter Keys_jangling
1  0013a1db.wav              Flute Oboe Violin_or_fiddle
2  002bb878.wav        Bass_drum Knock Computer_keyboard
3  002d392d.wav               Bass_drum Knock Snare_drum
4  00326aa9.wav                  Oboe Flute Glockenspiel
5  0038a046.wav           Knock Bass_drum Electric_piano
6  003995fa.wav                 Glockenspiel Squeak Meow
7  005ae625.wav      Acoustic_guitar Electric_piano Gong
8  007759c4.wav       Telephone Clarinet Acoustic_guitar
9  008afd93.wav                 Saxophone Clarinet Flute


In [5]:
df_ens = pd.DataFrame(df.label.str.split(' ',2).tolist(),columns=['1','2','3'])

df_ens = pd.merge(pd.DataFrame(df.fname),df_ens,how='inner',right_index=True,left_index=True)
df_ens

,fname,1,2,3
0,00063640.wav,Computer_keyboard,Shatter,Keys_jangling
1,0013a1db.wav,Flute,Oboe,Violin_or_fiddle
2,002bb878.wav,Bass_drum,Knock,Computer_keyboard
3,002d392d.wav,Bass_drum,Knock,Snare_drum
4,00326aa9.wav,Oboe,Flute,Glockenspiel
5,0038a046.wav,Knock,Bass_drum,Electric_piano
6,003995fa.wav,Glockenspiel,Squeak,Meow
7,005ae625.wav,Acoustic_guitar,Electric_piano,Gong
8,007759c4.wav,Telephone,Clarinet,Acoustic_guitar
9,008afd93.wav,Saxophone,Clarinet,Flute


## NN predict

In [6]:
model1 = load_model('model/strong_S1P2_NNclf_41dim_softmax.h5')
model2 = load_model('model/strong_S1P2_NNclf_41dim.h5')
ans1 = model1.predict(test_X)
ans2 = model2.predict(test_X)
ans_NN = (ans1+ans2)/2

In [7]:
df_nn = pd.DataFrame(np.argmax(ans_NN,axis=1),columns=['nn1'])
df_nn['nn1_ch'] = df_nn['nn1'].map(reverse_dict)
df_nn = df_nn.filter(['nn1_ch'])
df_nn

,nn1_ch
0,Keys_jangling
1,Flute
2,Bass_drum
3,Bass_drum
4,Oboe
5,Knock
6,Squeak
7,Acoustic_guitar
8,Telephone
9,Saxophone


## Merge Prediction

In [62]:

# 1 => nn => 2 => 3
def que(x): # decide fin2
    if x['nn1_ch'] == x['1'] :
        return x['2']
    else:
        return x['nn1_ch']
def que2(x): # decide fin3
    if x['nn1_ch'] == x['1'] :
        return x['3']
    else:
        return x['2']

In [8]:
# better
# nn1_ch => 1 => 2 => 3
def que(x): # decide fin2
    if x['nn1_ch'] == x['1'] :
        return x['2']
    else:
        return x['1']
def que2(x): # decide fin3
    if x['nn1_ch'] == x['1'] :
        return x['3']
    else:
        return x['2']

In [9]:
df_ans = pd.merge(df_ens,df_nn,how='inner',left_index=True,right_index=True)
df_ans['fin2'] = df_ans.apply(que,axis=1)
df_ans['fin3'] = df_ans.apply(que2,axis=1)
df_ans['final'] = df_ans['nn1_ch']+' '+df_ans['fin2']+' '+df_ans['fin3'] # nn=>1=>2=>3
# df_ans['final'] = df_ans['1']+' '+df_ans['fin2']+' '+df_ans['fin3'] # 1=>nn=>2=>3
dfF = df_ans.filter(['fname','final'])
dfF.columns = ['fname','label']
dfF

,fname,label
0,00063640.wav,Keys_jangling Computer_keyboard Shatter
1,0013a1db.wav,Flute Oboe Violin_or_fiddle
2,002bb878.wav,Bass_drum Knock Computer_keyboard
3,002d392d.wav,Bass_drum Knock Snare_drum
4,00326aa9.wav,Oboe Flute Glockenspiel
5,0038a046.wav,Knock Bass_drum Electric_piano
6,003995fa.wav,Squeak Glockenspiel Squeak
7,005ae625.wav,Acoustic_guitar Electric_piano Gong
8,007759c4.wav,Telephone Clarinet Acoustic_guitar
9,008afd93.wav,Saxophone Clarinet Flute


In [10]:
dfF.to_csv('result/ans_2nnclf41dim_ori23.csv',index=False)